In [260]:
#inception模型
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import Conv2d,MaxPool2d,Flatten,Linear
import matplotlib.pyplot as plt
import warnings
from typing import Callable, Any, Optional, Tuple, List
from keras.datasets import mnist
import torch.optim as optim
from keras.utils import to_categorical

In [199]:
(train_data,test_data)=mnist.load_data()

In [171]:

class BasicConv2d(nn.Module):
    def __init__(self, in_channels: int, out_channels: int,**kwargs:Any) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels,**kwargs,bias=True)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        #x为tensor 输出也为tensor
        x1 = self.conv(x)
        x2 = self.bn(x1)
        return F.relu(x2, inplace=True)


In [272]:
class Inception(nn.Module):
    def __init__(self, in_channels: int, pool_features: int, conv_block: Optional[Callable[..., nn.Module]] = None):
        super().__init__()
        if conv_block is None:
            conv_block = BasicConv2d
        self.branch1x1 = conv_block(in_channels, 64, kernel_size=1)

        self.branch5x5_1 = conv_block(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = conv_block(48, 64, kernel_size=5, padding=2)
    
        self.branch3x3dbl_1 = conv_block(in_channels, 64, kernel_size=1)
        self.branch3x3dbl_2 = conv_block(64, 96, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = conv_block(96, 96, kernel_size=3, padding=1)        
        self.branch_pool = conv_block(in_channels, pool_features, kernel_size=1)
        self.flatten1=Flatten()
        self.Dense1=Linear(225792,10)
    ###############
    #前向传播
    def forward(self,x):
       branch1x1 = self.branch1x1(x)
       branch5x5 = self.branch5x5_1(x)
       branch5x5 = self.branch5x5_2(branch5x5)
       branch3x3dbl = self.branch3x3dbl_1(x)
       branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
       branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)
       branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
       branch_pool = self.branch_pool(branch_pool)
       hidden = torch.concat([branch1x1, branch5x5, branch3x3dbl, branch_pool],axis=1)
       hidden_output=self.flatten1(hidden)
       output=F.softmax(self.Dense1(hidden_output))
       
       return output


#训练模型
inception=Inception(in_channels=1,pool_features=64)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(inception.parameters(),lr=0.1,betas=(0.9,0.98))
train_x,train_y = train_data
train_x=torch.tensor(train_x,dtype=torch.float32).reshape(60000,1,28,28)
train_y=torch.tensor(to_categorical(train_y))
for epoches in range(1):
    for samples_index in range(len(train_x)):
        y_pre=inception(train_x[samples_index].reshape(1,1,28,28))
        loss=criterion(y_pre[0],train_y[samples_index])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
